# Thermal Neural Networks (Tensorflow example)

This jupyter notebook showcases how to utilize a [thermal neural network (TNN)](https://www.sciencedirect.com/science/article/pii/S0952197622005279) on an exemplary data set with the [Tensorflow](https://www.tensorflow.org/) framework.

This example is concise for the sake of comprehensibility, that is, no cross-validation with a validation set is conducted, e.g., for early stopping, no learning rate scheduling, no repeated experiments with different random number generator seeds, etc.

Feel free to build and expand your training pipeline on top of this example.

The data set can be downloaded from [Kaggle](https://www.kaggle.com/wkirgsn/electric-motor-temperature).
It should be placed in `data/input/`.

In [2]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import tensorflow as tf